In [25]:
import boto3
from botocore.exceptions import ClientError
import time

class BedrockAgentService:
    """
    A service class to interact with AWS Bedrock Agents.
    
    This class provides functionality to invoke Bedrock agents and handle their responses,
    including parameter parsing and control flow management.
    """
    
    def __init__(self, agent_id: str, alias_id: str = "TSTALIASID", session_id = None, client = None) -> None:
        """
        Initialize the BedrockAgentService.
        
        Args:
            agent_id (str): The ID of the Bedrock agent to interact with
            alias_id (str, optional): The alias ID for the agent. Defaults to "TSTALIASID"
            client: The boto3 client for Bedrock runtime. If None, creates a new client
        """
        self.agents_runtime_client = (
            client if client else boto3.client("bedrock-agent-runtime")
        )
        self.agent_id = agent_id
        self.alias_id = alias_id
        self.session_id = session_id or str(int(time.time()))

    def set_session_id(self, session_id: str) -> None:
        """
        Set the session ID for the agent.

        Args:
            session_id (str): The session ID to be used for the agent interaction
        """
        self.session_id = session_id

    def parse_parameters(self, parameters: list) -> dict:
        """
        Parse parameters from the agent response into a dictionary format.
        
        Args:
            parameters (list): List of parameter dictionaries containing name and value pairs
            
        Returns:
            dict: Dictionary mapping parameter names to their values
        """
        return {param["name"]: param["value"] for param in parameters}

    def return_control(self, completion: dict) -> dict:
        """
        Process the completion response and extract control flow information.
        
        Args:
            completion (dict): The completion response from the agent containing control information
            
        Returns:
            dict: Processed control information including invocation details and parameters
        """
        rc = completion.get("returnControl", {})
        invocationId = rc.get("invocationId", "")
        invocationInputs = rc.get("invocationInputs",[])

        return_dict = { "invocationId": invocationId}
        
        if len(invocationInputs):
            functionInvocationInput = invocationInputs[0].get("functionInvocationInput", {})
            actionGroup = functionInvocationInput.get("actionGroup", {})
            agentId = functionInvocationInput.get("agentId", None)
            actionInvocationType = functionInvocationInput.get("actionInvocationType", "")
            functionName = functionInvocationInput.get("function", "")
            parameters = functionInvocationInput.get("parameters", {})
            parsed_parameters = self.parse_parameters(parameters)
            return_dict.update(
                {
                    "actionGroup": actionGroup,
                    "agentId": agentId,
                    "functionName": functionName,
                    "parameters": parsed_parameters,
                    "actionInvocationType": actionInvocationType,
                }
            )
        return return_dict

    def return_control_invocation_results(
        self, 
        invocation_id: str, 
        action_group: str, 
        function_name: str, 
        invocation_result: str,
        agent_id : str  = None,
        session_id: str = None, 
    ) -> str:
        """
        Send the results of a function invocation back to the agent.
        
        Args:
            session_id (str): The current session identifier
            invocation_id (str): The ID of the invocation being responded to
            action_group (str): The action group that was invoked
            function_name (str): The name of the function that was called
            invocation_result (str): The result of the function invocation
            
        Returns:
            str: The completion response from the agent after processing the results
        """
        try:
            response = self.agents_runtime_client.invoke_agent(
                agentId=self.agent_id,
                agentAliasId=self.alias_id,
                sessionId=session_id or self.session_id,
                sessionState={
                    "invocationId": invocation_id,
                    "returnControlInvocationResults": [
                        {
                            "functionResult": {
                                "actionGroup": action_group,
                                "agentId": agent_id or self.agent_id,
                                "function": function_name,
                                "responseBody": {"TEXT": {"body": invocation_result}},
                            }
                        }
                    ],
                },
            )

            completion = ""

            for event in response.get("completion"):
                if event.get("returnControl"):
                    return self.return_control(event)
                chunk = event["chunk"]
                completion = completion + chunk["bytes"].decode()

        except ClientError as e:
            print(f"Couldn't invoke agent. {e}")

        return completion

    # from https://docs.aws.amazon.com/code-library/latest/ug/python_3_bedrock-agent-runtime_code_examples.html
    def invoke_agent(self, prompt: str, session_id: str = None) -> dict:
        """
        Invoke the Bedrock agent with a prompt and handle the response.
        
        Args:
            session_id (str): The session identifier for the conversation
            prompt (str): The user's input prompt to send to the agent
            
        Returns:
            dict: The processed response from the agent including any control flow modifications
        """

        kwargs = dict(
            agentId=self.agent_id,
            agentAliasId=self.alias_id,
            sessionId=session_id or self.session_id,
            inputText=prompt
        )
        if prompt.startswith("/new "):            
            kwargs["inputText"] = prompt.replace("/new ", "")
            kwargs["endSession"] = True

        print ("AGENT INVOCATION KWARGS")
        print (kwargs)
        try:
            response = self.agents_runtime_client.invoke_agent(**kwargs, )

            completion = ""

            for event in response.get("completion"):
                if event.get("returnControl"):
                    return event, self.return_control(event)
                chunk = event["chunk"]
                completion = completion + chunk["bytes"].decode()

        except ClientError as e:
            print(f"Couldn't invoke agent. {e}")
            return None

        return completion


In [27]:
AGENT_ID = "859XC0IM9C"
AGENT_ALIAS_ID = "TSTALIASID"
session_id = "004"
bedrock_agent = BedrockAgentService(AGENT_ID, AGENT_ALIAS_ID, session_id)
first_utternace = "Hola tengo un problema con mi pedido , mi rut es 10192797-1 y el pedido 10026656"
bedrock_agent.invoke_agent(first_utternace)

AGENT INVOCATION KWARGS
{'agentId': '859XC0IM9C', 'agentAliasId': 'TSTALIASID', 'sessionId': '004', 'inputText': 'Hola tengo un problema con mi pedido , mi rut es 10192797-1 y el pedido 10026656'}


'Según nuestro sistema, su pedido número 10026656 se encuentra actualmente en estado "Despacho Programado". Está programado para ser entregado el 6 de agosto de 2024 en la dirección Calle las acacias 123, Providencia, Región Metropolitana. ¿En qué más puedo ayudarle?'

In [28]:
bedrock_agent.invoke_agent("Pero es que ya pasó la fecha de envío")

AGENT INVOCATION KWARGS
{'agentId': '859XC0IM9C', 'agentAliasId': 'TSTALIASID', 'sessionId': '004', 'inputText': 'Pero es que ya pasó la fecha de envío'}


'He creado un ticket de soporte con el número 202502170225 para investigar el retraso en su pedido. Nuestro equipo revisará el estado de su envío y se comunicará con usted pronto para resolver la situación. El ticket está actualmente abierto y en proceso de investigación. ¿Hay algún detalle adicional que pueda proporcionarme sobre el pedido?'

In [29]:
bedrock_agent.invoke_agent("Puedes ayudarme a escalarlo con alguien humano?")

AGENT INVOCATION KWARGS
{'agentId': '859XC0IM9C', 'agentAliasId': 'TSTALIASID', 'sessionId': '004', 'inputText': 'Puedes ayudarme a escalarlo con alguien humano?'}


({'returnControl': {'invocationId': 'bf5b5069-c9c4-49d3-bb0a-cf6197702ab4',
   'invocationInputs': [{'functionInvocationInput': {'actionGroup': 'Escalation',
      'actionInvocationType': 'RESULT',
      'agentId': 'OELO0POXNZ',
      'collaboratorName': 'escalation',
      'function': 'escalate',
      'parameters': [{'name': 'identity_document_number',
        'type': 'string',
        'value': '10192797-1'},
       {'name': 'order_number', 'type': 'string', 'value': '10026656'},
       {'name': 'description',
        'type': 'string',
        'value': 'Cliente reporta retraso en la entrega de su pedido. La fecha de envío programada ya pasó y no ha recibido su producto.'},
       {'name': 'ticket_number',
        'type': 'string',
        'value': '202502170225'}]}}]}},
 {'invocationId': 'bf5b5069-c9c4-49d3-bb0a-cf6197702ab4',
  'actionGroup': 'Escalation',
  'agentId': 'OELO0POXNZ',
  'functionName': 'escalate',
  'parameters': {'identity_document_number': '10192797-1',
   'order_nu

In [ ]:
bedrock_agent.return_control_invocation_results(
    "bf5b5069-c9c4-49d3-bb0a-cf6197702ab4",
    "Escalation",
    "escalate",
    "escalation=OK",
    "OELO0POXNZ",
)

'He escalado su caso a nuestro equipo de soporte especializado. Un agente humano revisará su pedido 10026656 y se comunicará con usted pronto para resolver el problema del retraso en la entrega. Le pedimos disculpas por las molestias ocasionadas.'

In [33]:
bedrock_agent.invoke_agent("OK Gracias, por tu trabajo")

AGENT INVOCATION KWARGS
{'agentId': '859XC0IM9C', 'agentAliasId': 'TSTALIASID', 'sessionId': '004', 'inputText': 'OK Gracias, por tu trabajo'}


'Muchas gracias por su comprensión. Estamos comprometidos en resolver su problema y le aseguramos que nuestro equipo hará todo lo posible para atender su caso. Que tenga un excelente día.'